In [1]:
import pandas as pd

In [8]:
# load in the sample data
airnow_df = pd.read_csv("../../Data/Raw/airnow_sample_9_23_25.csv")
aqs_df = pd.read_csv("../../Data/Raw/aqs_sample_9_24_25.csv")
hvo_df = pd.read_csv("../../Data/Raw/hvo_sample_9_24_25.csv")
open_meteo_df = pd.read_csv("../../Data/Raw/openmeteo_sample_9_24_25.csv")
purpleair_df = pd.read_csv("../../Data/Raw/purpleair_sample_9_24_25.csv")

quick loop to print out a basic EDA report for each data frame 

In [23]:
dfs = {
    "airnow": airnow_df,
    "aqs": aqs_df,
    "hvo": hvo_df,
    "openmeteo": open_meteo_df,
    "purpleair": purpleair_df
}
for name, df in dfs.items():
    print(f"{name} size: {df.shape}\n")
    print(f"{name} columns:")
    print(list(df.columns), "\n")
    print(f"{name} datatypes and nulls:")
    print(df.info())
    print("--------------------------------------------")

airnow size: (1, 10)

airnow columns:
['DateObserved', 'HourObserved', 'LocalTimeZone', 'ReportingArea', 'StateCode', 'Latitude', 'Longitude', 'ParameterName', 'AQI', 'Category'] 

airnow datatypes and nulls:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   DateObserved   1 non-null      object 
 1   HourObserved   1 non-null      int64  
 2   LocalTimeZone  1 non-null      object 
 3   ReportingArea  1 non-null      object 
 4   StateCode      1 non-null      object 
 5   Latitude       1 non-null      float64
 6   Longitude      1 non-null      float64
 7   ParameterName  1 non-null      object 
 8   AQI            1 non-null      int64  
 9   Category       1 non-null      object 
dtypes: float64(2), int64(2), object(6)
memory usage: 212.0+ bytes
None
--------------------------------------------
aqs size: (6813, 29)

aqs columns:
['state_co